In [3]:
import pandas as pd

train_path = '/content/data/train.csv'
test_path = '/content/data/test.csv'
#xieyuan那边处理之后的结果
train_newmake_path = '/content/data/train_newmake.csv'

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_train_newmake = pd.read_csv(train_newmake_path)

一些问题：  

1.很多make-model的销售数量非常少，无法基于某一个model去除outlier。直接在所有数据上面去除outlier是否可行？

2.在模型训练与预测时检测这些销售数量少的model的预测误差，检查这些销售数量少的model是否会出现价格波动很大的问题（导致这部分价格的预测不准确）

3.power, engine_cap, road_tap这一组feature的处理。power对于price的影响最大，接着是engine_cap，最后是road_tap。这里面road_tap与engine_cap相关性非常高（0.95），可以尝试忽略road_tap.(尝试了利用线性回归去基于road_tap与engine_cap中的某一个去预测另一个，可以用于后续的缺失值填补。但平均误差还是不小，另外在power的的缺失值删除后，这俩特征的缺失值数量不多(一共400)，感觉没有必要引入缺失值填补的处理？)

4.mileage的处理（有一定作用，但是缺失值特别多，暂时是删除。）

5.coe omv arf depreciation这一组features的处理，引入depreciation/coe反映coe剩余时间，深处coe

In [4]:
#display(df_train_newmake.head())
df_train_newmake.columns

Index(['listing_id', 'title', 'make', 'model', 'make.bak', 'model.bak',
       'description', 'manufactured', 'original_reg_date', 'reg_date',
       'type_of_vehicle', 'category', 'transmission', 'curb_weight', 'power',
       'fuel_type', 'engine_cap', 'no_of_owners', 'depreciation', 'coe',
       'road_tax', 'dereg_value', 'mileage', 'omv', 'arf', 'opc_scheme',
       'lifespan', 'eco_category', 'features', 'accessories',
       'indicative_price', 'price'],
      dtype='object')

In [5]:
grouped_df = df_train_newmake.groupby(['make', 'model']).size().reset_index(name='count')
#display(grouped_df)

In [6]:
grouped_df_less_than_10 = grouped_df[grouped_df['count'] < 10]
print(grouped_df.shape[0])
print(grouped_df_less_than_10.shape[0])
#这里很多model的数量非常少，outlier不好去除。

1031
641


In [7]:
features = ['make' , 'model' , 'manufactured'	, 'reg_date', 'type_of_vehicle' ,'category','transmission','curb_weight','power',
                      'engine_cap','no_of_owners','depreciation','coe','road_tax','dereg_value','mileage','omv','arf','price']
df = df_train_newmake[features].copy()

In [8]:
for column in df.columns:
    print(f"Column: {column}")
    print(df[column].apply(type).unique())
#数据的type都没有问题

Column: make
[<class 'str'>]
Column: model
[<class 'str'>]
Column: manufactured
[<class 'float'>]
Column: reg_date
[<class 'str'>]
Column: type_of_vehicle
[<class 'str'>]
Column: category
[<class 'str'>]
Column: transmission
[<class 'str'>]
Column: curb_weight
[<class 'float'>]
Column: power
[<class 'float'>]
Column: engine_cap
[<class 'float'>]
Column: no_of_owners
[<class 'float'>]
Column: depreciation
[<class 'float'>]
Column: coe
[<class 'float'>]
Column: road_tax
[<class 'float'>]
Column: dereg_value
[<class 'float'>]
Column: mileage
[<class 'float'>]
Column: omv
[<class 'float'>]
Column: arf
[<class 'float'>]
Column: price
[<class 'float'>]


In [9]:
missing_values = df.isnull().sum()
print(missing_values)

make                  0
model                 0
manufactured          7
reg_date              0
type_of_vehicle       0
category              0
transmission          0
curb_weight         307
power              2640
engine_cap          596
no_of_owners         18
depreciation        507
coe                   0
road_tax           2632
dereg_value         220
mileage            5304
omv                  64
arf                 174
price                 0
dtype: int64


In [14]:
#这几个feature都是有作用的，尝试将其有缺失值的列都删掉，在后续的训练中进行对比，数据量少的非常多，大概率不可行
print(df.shape)
df_delete = df.dropna()
print(df_delete.shape)

(25000, 19)
(17982, 19)


In [15]:
#power与road_tax及
df_delete_1 = df.drop(['mileage','road_tax'], axis=1)
df_delete_1.dropna(inplace=True)
print(df_delete_1.shape)


(21433, 17)


In [10]:
#curb_weight这个属性不能由车的类别（model）来决定，但与price的相关性不大
curb_weight_stats = df.groupby(['make', 'model'])['curb_weight'].agg(['mean', 'min', 'max', 'count']).reset_index()
print(curb_weight_stats)
curb_weight_stats['difference'] = curb_weight_stats['max'] - curb_weight_stats['min']
# 找出差值最大的前十个类别
top_10_diff = curb_weight_stats.nlargest(10, 'difference')
print('差值最大的十个类别\n',top_10_diff)
selected_columns_campare = ['price','curb_weight']
correlation_matrix = df[selected_columns_campare].corr()
print('相关性\n',correlation_matrix)

            make    model          mean      min      max  count
0           aion        y   1750.000000   1750.0   1750.0      1
1     alfa romeo      159   1490.000000   1490.0   1490.0      3
2     alfa romeo     2000           NaN      NaN      NaN      0
3     alfa romeo       4c    920.000000    920.0    920.0      1
4     alfa romeo   giulia   1583.444444   1429.0   1695.0      9
...          ...      ...           ...      ...      ...    ...
1026       volvo     xc90   2064.833333   1997.0   2222.0      6
1027       volvo  xc90 t5   2046.833333   2037.0   2094.0     18
1028       volvo  xc90 t6   2093.272727   2077.0   2099.0     11
1029      yutong  zk6117h  11226.666667  11200.0  11260.0      3
1030      yutong  zk6119h  11720.000000  11720.0  11720.0      1

[1031 rows x 6 columns]
差值最大的十个类别
            make      model          mean      min      max  count  difference
702  mitsubishi       fuso   8302.800000   2500.0  17520.0     50     15020.0
859      scania       p360  

In [11]:
power_stats = df.groupby(['make', 'model'])['power'].agg(['mean', 'min', 'max', 'count']).reset_index()
power_stats['difference'] = power_stats['max'] - power_stats['min']
# 找出差值最大的前十个类别
top_10_diff = power_stats.nlargest(10, 'difference')
print('差值最大的十个类别\n',top_10_diff)
selected_columns_campare = ['price','power']
correlation_matrix = df[selected_columns_campare].corr()
print('相关性\n',correlation_matrix)

差值最大的十个类别
               make         model        mean    min    max  count  difference
562  mercedes-benz           amg  229.632479   90.0  484.0    117       394.0
825        porsche        taycan  351.029412  240.0  525.0     34       285.0
472     land rover   range rover  218.471074  132.0  415.0    121       283.0
414         jaguar        f-pace  218.862069  132.0  405.0     29       273.0
468     land rover  defender 110  222.866667   44.0  307.0     15       263.0
915          tesla       model 3  292.156250  208.0  461.0     64       253.0
949         toyota  land cruiser  231.363636   60.0  305.0     11       245.0
217            byd          seal  230.000000  150.0  390.0      8       240.0
4       alfa romeo        giulia  274.555556  147.0  375.0      9       228.0
9       alfa romeo       stelvio  295.800000  148.0  375.0      5       227.0
相关性
           price     power
price  1.000000  0.703498
power  0.703498  1.000000


In [12]:
selected_columns_campare = ['price','power','engine_cap','road_tax','mileage']
correlation_matrix = df[selected_columns_campare].corr()
print('相关性\n',correlation_matrix)

相关性
                price     power  engine_cap  road_tax   mileage
price       1.000000  0.703498    0.443110  0.527713 -0.392547
power       0.703498  1.000000    0.865975  0.822907 -0.230732
engine_cap  0.443110  0.865975    1.000000  0.946788 -0.035450
road_tax    0.527713  0.822907    0.946788  1.000000 -0.051804
mileage    -0.392547 -0.230732   -0.035450 -0.051804  1.000000


In [16]:
selected_columns_campare = ['engine_cap','road_tax','price','power']
correlation_matrix = df_delete[selected_columns_campare].corr()
print('相关性\n',correlation_matrix)

相关性
             engine_cap  road_tax     price     power
engine_cap    1.000000  0.958415  0.624605  0.880568
road_tax      0.958415  1.000000  0.537531  0.818795
price         0.624605  0.537531  1.000000  0.714739
power         0.880568  0.818795  0.714739  1.000000


In [27]:
df_map_taxcap = df.dropna(subset=['engine_cap', 'road_tax']) #x = engine_cap
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# 特征和目标变量
X = df_map_taxcap[['engine_cap']]
y = df_map_taxcap['road_tax']
# 按 4:1 比例划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 建立线性回归模型并训练
model = LinearRegression()
model.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = model.predict(X_test)

# 计算模型的均方误差（MSE）
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# 输出测试集上的预测结果和真实值
print("Predicted vs Actual (First 10):")
for pred, actual in zip(y_pred[:10], y_test[:10]):
    print(f"Predicted: {pred}, Actual: {actual}")


Mean Squared Error: 165445.72749010028
Predicted vs Actual (First 10):
Predicted: 543.8628942768983, Actual: 744.0
Predicted: 3000.091732919246, Actual: 2382.0
Predicted: 705.9801152582736, Actual: 682.0
Predicted: 8739.959000307, Actual: 8250.0
Predicted: 696.8036687876297, Actual: 1020.0
Predicted: 705.9801152582736, Actual: 682.0
Predicted: 433.74553662917174, Actual: 578.0
Predicted: 860.4502975141127, Actual: 1039.0
Predicted: 652.4508441795178, Actual: 1048.0
Predicted: 543.8628942768983, Actual: 682.0


In [ ]:
missing_values_test = df_test.isnull().sum()
print(missing_values_test)

listing_id               0
title                    0
make                   541
model                    0
description            268
manufactured             3
original_reg_date     9903
reg_date                 0
type_of_vehicle          0
category                 0
transmission             0
curb_weight            110
power                 1086
fuel_type             7650
engine_cap             235
no_of_owners             8
depreciation           201
coe                      0
road_tax              1082
dereg_value             83
mileage               2166
omv                     29
arf                     65
opc_scheme            9953
lifespan              9026
eco_category             0
features               323
accessories           1566
indicative_price     10000
dtype: int64


In [17]:
# 提取年份并存储在新的列中
df['reg_date_trans'] = pd.to_datetime(df['reg_date'], infer_datetime_format=True, errors='coerce')

# 提取年份和月份
df['year'] = df['reg_date_trans'].dt.year
df['month'] = df['reg_date_trans'].dt.month
display(df)

<ipython-input-17-cd3be550f657>:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['reg_date_trans'] = pd.to_datetime(df['reg_date'], infer_datetime_format=True, errors='coerce')


,make,model,manufactured,reg_date,type_of_vehicle,category,transmission,curb_weight,power,engine_cap,...,coe,road_tax,dereg_value,mileage,omv,arf,price,reg_date_trans,year,month
0,land rover,range rover,2018.0,2018-03-08,suv,parf car,auto,1884.0,280.0,2995.0,...,48011.0,2380.0,103323.0,96000.0,88906.0,132031.0,193788.0,2018-03-08,2018,3
1,mercedes-benz,c200,2017.0,2017-12-28,luxury sedan,"parf car, premium ad car",auto,1465.0,135.0,1991.0,...,47002.0,1202.0,45179.0,85680.0,40678.0,43950.0,96800.0,2017-12-28,2017,12
2,honda,odyssey,2007.0,2007-09-19,mpv,"coe car, premium ad car, low mileage car",auto,1648.0,118.0,2354.0,...,50355.0,2442.0,16003.0,138000.0,27994.0,30794.0,39800.0,2007-09-19,2007,9
3,toyota,corolla altis,2008.0,2008-12-15,mid-sized sedan,"coe car, premium ad car",auto,1195.0,80.0,1598.0,...,27571.0,1113.0,12184.0,160000.0,16084.0,16084.0,44800.0,2008-12-15,2008,12
4,lexus,gs300,2006.0,2006-12-22,luxury sedan,"coe car, premium ad car",auto,1660.0,183.0,2995.0,...,48479.0,3570.0,9138.0,183000.0,50414.0,55456.0,25800.0,2006-12-22,2006,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,mercedes-benz,c180,2011.0,2011-07-14,luxury sedan,"coe car, premium ad car, low mileage car",auto,1500.0,115.0,1597.0,...,45272.0,1039.0,30921.0,94000.0,31019.0,31019.0,85000.0,2011-07-14,2011,7
24996,bentley,bentayga,2018.0,2020-04-29,suv,"parf car, direct owner sale, rare & exotic",auto,2470.0,404.0,3996.0,...,37500.0,3944.0,NaN,73838.0,137136.0,218845.0,412888.0,2020-04-29,2020,4
24997,mercedes-benz,a180,2014.0,2015-04-01,hatchback,"parf car, premium ad car, low mileage car, sgc...",auto,1395.0,90.0,1595.0,...,61410.0,740.0,11890.0,88388.0,22897.0,14056.0,23888.0,2015-04-01,2015,4
24998,audi,a3,2017.0,2017-06-30,hatchback,parf car,auto,1275.0,85.0,999.0,...,46489.0,392.0,24070.0,94900.0,23772.0,15281.0,53300.0,2017-06-30,2017,6


In [18]:
min_year = df['year'].min()
df['month_order']=df['month']+12*(df['year']-min_year)

In [19]:
selected_columns_campare = ['year','month_order','month','price']
correlation_matrix = df[selected_columns_campare].corr()
print('相关性\n',correlation_matrix)

相关性
                  year  month_order     month     price
year         1.000000     0.998516 -0.026764  0.235423
month_order  0.998516     1.000000  0.027725  0.234360
month       -0.026764     0.027725  1.000000 -0.019396
price        0.235423     0.234360 -0.019396  1.000000


In [20]:
df['de/coe']=df['depreciation'] / df['coe']

In [21]:
df.columns

Index(['make', 'model', 'manufactured', 'reg_date', 'type_of_vehicle',
       'category', 'transmission', 'curb_weight', 'power', 'engine_cap',
       'no_of_owners', 'depreciation', 'coe', 'road_tax', 'dereg_value',
       'mileage', 'omv', 'arf', 'price', 'reg_date_trans', 'year', 'month',
       'month_order', 'de/coe'],
      dtype='object')

In [22]:
df_clean = df.drop(['mileage','road_tax','reg_date','coe','reg_date_trans', 'month','month_order',], axis=1)
df_clean.dropna(inplace=True)
print(df_clean.shape)
df_clean.columns

(21433, 17)


Index(['make', 'model', 'manufactured', 'type_of_vehicle', 'category',
       'transmission', 'curb_weight', 'power', 'engine_cap', 'no_of_owners',
       'depreciation', 'dereg_value', 'omv', 'arf', 'price', 'year', 'de/coe'],
      dtype='object')

In [ ]:
display(df_clean)

In [23]:
categorical = ['make', 'model','type_of_vehicle','category','transmission']
unique_counts = df_clean[categorical].nunique()

# 输出统计结果
print(unique_counts)

make                63
model              781
type_of_vehicle     10
category           165
transmission         2
dtype: int64


In [30]:
numerical = ['manufactured', 'curb_weight', 'power', 'engine_cap', 'no_of_owners',
       'depreciation', 'dereg_value', 'omv', 'arf', 'year', 'de/coe']

In [31]:
x = df_clean[['make', 'model', 'manufactured', 'type_of_vehicle', 'category',
       'transmission', 'curb_weight', 'power', 'engine_cap', 'no_of_owners',
       'depreciation', 'dereg_value', 'omv', 'arf', 'year','de/coe']].copy()
y = df_clean['price'].copy()

In [32]:
#决策树与随机森林（这两个模型对于numerical特征不需要归一化，categorical特征直接使用label）
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
x_1 = x.copy()
y_1 = y.copy()
# 将 categorical 特征进行 Label Encoding
label_encoders = {}
for col in categorical:
    le = LabelEncoder()
    x_1[col] = le.fit_transform(x_1[col])
    label_encoders[col] = le  # 保存 LabelEncoder 对象，方便以后逆转编码
x_train, x_test, y_train, y_test = train_test_split(x_1, y_1, test_size=0.2, random_state=42)

In [33]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# 初始化决策树回归模型
tree_model = DecisionTreeRegressor(max_depth=13,random_state=42)

# 训练模型
tree_model.fit(x_train, y_train)

# 预测
y_pred_tree = tree_model.predict(x_test)

# 评估模型性能（均方误差）
rmse_tree = mean_squared_error(y_test, y_pred_tree, squared=False)
print(f"Decision Tree RMSE: {rmse_tree}")
r2_tree = r2_score(y_test, y_pred_tree)
print(f"Decision Tree R² Score: {r2_tree}")


Decision Tree RMSE: 30707.96067654791
Decision Tree R² Score: 0.9514734905629371


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [34]:
for pred, actual in zip(y_pred_tree[:10], y_test[:10]):
    print(f"Predicted: {pred}, Actual: {actual}")

Predicted: 94954.0255319149, Actual: 92800.0
Predicted: 82196.0, Actual: 79800.0
Predicted: 51284.05882352941, Actual: 47800.0
Predicted: 2000000.0, Actual: 2350000.0
Predicted: 150150.0, Actual: 210000.0
Predicted: 29607.418604651164, Actual: 31800.0
Predicted: 99673.68656716419, Actual: 99333.0
Predicted: 120148.5, Actual: 117800.0
Predicted: 46965.678899082566, Actual: 45300.0
Predicted: 63638.69014084507, Actual: 72800.0


In [35]:
print(f"Tree depth: {tree_model.get_depth()}")

Tree depth: 13


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# 初始化随机森林回归模型
forest_model = RandomForestRegressor(n_estimators=200,max_depth=20, random_state=42)

# 训练模型
forest_model.fit(x_train, y_train)

# 预测
y_pred_forest = forest_model.predict(x_test)

# 评估模型性能（均方误差）
mse_forest = mean_squared_error(y_test, y_pred_forest)
print(f"Random Forest Mean Squared Error: {mse_forest}")
rmse_forest = mean_squared_error(y_test, y_pred_forest, squared=False)
print(f"Decision Tree RMSE: {rmse_forest}")
r2_forest = r2_score(y_test, y_pred_forest)
print(f"Random Forest R² Score: {r2_forest}")

Random Forest Mean Squared Error: 631170245.2604108
Decision Tree RMSE: 25123.101824026642
Random Forest R² Score: 0.9675194317472252


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
for pred, actual in zip(y_pred_forest[:10], y_test[:10]):
    print(f"Predicted: {pred}, Actual: {actual}")

Predicted: 88186.11656442819, Actual: 92800.0
Predicted: 86941.29725531315, Actual: 79800.0
Predicted: 49654.21970613942, Actual: 47800.0
Predicted: 2089948.196, Actual: 2350000.0
Predicted: 148418.06660967757, Actual: 210000.0
Predicted: 30280.193252661746, Actual: 31800.0
Predicted: 101412.05691526731, Actual: 99333.0
Predicted: 121196.93981092428, Actual: 117800.0
Predicted: 46351.599826239966, Actual: 45300.0
Predicted: 72406.74257920419, Actual: 72800.0


In [36]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
x_2 = x.copy()
y_2 = y.copy()
categorical_cols = ['transmission', 'type_of_vehicle','make']
x_categorical = x_2[categorical_cols]
# 初始化 OneHotEncoder
one_hot_encoder = OneHotEncoder(drop='first', sparse_output=False)  # drop='first' to avoid multicollinearity

# 对 categorical 特征进行 one-hot 编码
x_one_hot_encoded = one_hot_encoder.fit_transform(x_categorical)

# 将编码后的特征转化为 DataFrame 并添加列名
encoded_cols = one_hot_encoder.get_feature_names_out(categorical_cols)
x_one_hot_df = pd.DataFrame(x_one_hot_encoded, columns=encoded_cols)
display(x_one_hot_df)

,transmission_manual,type_of_vehicle_hatchback,type_of_vehicle_luxury sedan,type_of_vehicle_mid-sized sedan,type_of_vehicle_mpv,type_of_vehicle_others,type_of_vehicle_sports car,type_of_vehicle_stationwagon,type_of_vehicle_suv,type_of_vehicle_truck,...,make_smart,make_ssangyong,make_subaru,make_suzuki,make_td,make_toyota,make_triumph,make_valiant,make_volkswagen,make_volvo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21428,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21429,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21430,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21431,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.0 MB/s eta 0:00:00


In [38]:
import category_encoders as ce

# Target Encoding for high-cardinality categorical features
target_encoder = ce.TargetEncoder(cols=['model', 'category'])
x_encoded = target_encoder.fit_transform(x_2, y_2)
display(x_encoded)

,make,model,manufactured,type_of_vehicle,category,transmission,curb_weight,power,engine_cap,no_of_owners,depreciation,dereg_value,omv,arf,year,de/coe
0,land rover,195127.806453,2018.0,suv,81229.486769,auto,1884.0,280.0,2995.0,2.0,34270.0,103323.0,88906.0,132031.0,2018,0.713795
1,mercedes-benz,116426.225808,2017.0,luxury sedan,88014.114082,auto,1465.0,135.0,1991.0,2.0,21170.0,45179.0,40678.0,43950.0,2017,0.450406
2,honda,82599.011328,2007.0,mpv,101281.891645,auto,1648.0,118.0,2354.0,3.0,12520.0,16003.0,27994.0,30794.0,2007,0.248635
3,toyota,55953.602151,2008.0,mid-sized sedan,77213.323373,auto,1195.0,80.0,1598.0,3.0,10140.0,12184.0,16084.0,16084.0,2008,0.367778
4,lexus,88920.517587,2006.0,luxury sedan,77213.323373,auto,1660.0,183.0,2995.0,6.0,13690.0,9138.0,50414.0,55456.0,2006,0.282390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24994,honda,76359.370062,2019.0,mid-sized sedan,88014.114082,auto,1237.0,92.0,1597.0,1.0,15560.0,31900.0,20031.0,20044.0,2019,0.423932
24995,mercedes-benz,122640.525693,2011.0,luxury sedan,101281.891645,auto,1500.0,115.0,1597.0,2.0,12440.0,30921.0,31019.0,31019.0,2011,0.274784
24997,mercedes-benz,90321.320003,2014.0,hatchback,94143.568923,auto,1395.0,90.0,1595.0,2.0,21290.0,11890.0,22897.0,14056.0,2015,0.346686
24998,audi,73048.038991,2017.0,hatchback,81229.486769,auto,1275.0,85.0,999.0,2.0,15010.0,24070.0,23772.0,15281.0,2017,0.322872


In [41]:
selected_columns_campare = ['model','category']
correlation_matrix = pd.concat([x_encoded[selected_columns_campare],y_2],axis=1).corr()
print('相关性\n',correlation_matrix)
#category是否需要移除？

相关性
              model  category     price
model     1.000000  0.564119  0.773512
category  0.564119  1.000000  0.689763
price     0.773512  0.689763  1.000000


In [43]:
from sklearn.preprocessing import StandardScaler
#对于线性回归采用标准化

# 数值特征列表
numerical_cols = ['manufactured', 'curb_weight', 'power', 'engine_cap', 'no_of_owners',
                  'depreciation', 'dereg_value', 'omv', 'arf', 'year', 'de/coe']

# 获取数值特征和经过 Target Encoding 的特征
x_numerical_and_target_encoded = pd.concat([x_2[numerical_cols], x_encoded[['model', 'category']]], axis=1)

# 标准化
scaler = StandardScaler()
x_standardized = scaler.fit_transform(x_numerical_and_target_encoded)

# 将标准化后的数据转换为 DataFrame
x_standardized_df = pd.DataFrame(x_standardized, columns=numerical_cols + ['model', 'category'])

# 合并 One-Hot 编码后的特征和标准化后的数值特征
x_final = pd.concat([x_one_hot_df.reset_index(drop=True), x_standardized_df.reset_index(drop=True)], axis=1)

# 查看最终的特征集
display(x_final)

,transmission_manual,type_of_vehicle_hatchback,type_of_vehicle_luxury sedan,type_of_vehicle_mid-sized sedan,type_of_vehicle_mpv,type_of_vehicle_others,type_of_vehicle_sports car,type_of_vehicle_stationwagon,type_of_vehicle_suv,type_of_vehicle_truck,...,engine_cap,no_of_owners,depreciation,dereg_value,omv,arf,year,de/coe,model,category
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.251014,-0.216807,0.654471,0.869878,0.939258,1.038126,0.401777,0.527356,1.243809,-0.318262
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.035808,-0.216807,0.007574,-0.044541,-0.021881,-0.051288,0.164715,-0.027463,0.164611,-0.250699
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.475170,0.457721,-0.419575,-0.503386,-0.274662,-0.214006,-2.205906,-0.452488,-0.299245,-0.118573
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.439865,0.457721,-0.537102,-0.563446,-0.512017,-0.395944,-1.968844,-0.201517,-0.664621,-0.358257
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.251014,2.481305,-0.361798,-0.611350,0.172148,0.091022,-2.442968,-0.381383,-0.212561,-0.358257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21428,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.441076,-0.891335,-0.269455,-0.253377,-0.433357,-0.346965,0.638839,-0.083230,-0.384806,-0.250699
21429,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.441076,-0.216807,-0.423525,-0.268773,-0.214376,-0.211223,-1.257658,-0.397406,0.249825,-0.118573
21430,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.443496,-0.216807,0.013500,-0.568070,-0.376240,-0.421027,-0.309410,-0.245946,-0.193353,-0.189659
21431,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.164874,-0.216807,-0.296615,-0.376518,-0.358802,-0.405875,0.164715,-0.296109,-0.430213,-0.318262


In [45]:
import statsmodels.api as sm
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

x_train, x_test, y_train, y_test = train_test_split(x_final, y_2, test_size=0.2, random_state=42)
#ols
# 在 OLS 中，手动添加常数项 (截距)
x_train_ols = sm.add_constant(x_train.reset_index(drop=True))
y_train = y_train.reset_index(drop=True)

# 使用 OLS 模型进行训练
ols_model = sm.OLS(y_train, x_train_ols).fit()

# 进行预测
x_test_ols = sm.add_constant(x_test.reset_index(drop=True))
y_pred_ols = ols_model.predict(x_test_ols)

# 计算 RMSE
rmse_ols = np.sqrt(mean_squared_error(y_test, y_pred_ols))
print(f"OLS RMSE: {rmse_ols}")


OLS RMSE: 38422.10648711058


In [46]:
lr_model = LinearRegression()

# 训练模型
lr_model.fit(x_train, y_train)

# 进行预测
y_pred_lr = lr_model.predict(x_test)

# 计算 RMSE
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
print(f"Linear Regression RMSE: {rmse_lr}")

Linear Regression RMSE: 38422.10648711058


In [48]:
#使用归一化，用于mlp
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# 数值特征列表
numerical_cols = ['manufactured', 'curb_weight', 'power', 'engine_cap', 'no_of_owners',
                  'depreciation', 'dereg_value', 'omv', 'arf', 'year', 'de/coe']

# 获取数值特征和经过 Target Encoding 的特征
x_numerical_and_target_encoded = pd.concat([x_2[numerical_cols], x_encoded[['model', 'category']]], axis=1)

# 归一化数值特征和 Target Encoding 特征
scaler = MinMaxScaler()
x_normalized = scaler.fit_transform(x_numerical_and_target_encoded)

# 将归一化后的数据转换为 DataFrame
x_normalized_df = pd.DataFrame(x_normalized, columns=numerical_cols + ['model', 'category'])

# 合并 One-Hot 编码后的特征和归一化后的数值特征
x_final_1 = pd.concat([x_one_hot_df.reset_index(drop=True), x_normalized_df.reset_index(drop=True)], axis=1)

display(x_final_1)

,transmission_manual,type_of_vehicle_hatchback,type_of_vehicle_luxury sedan,type_of_vehicle_mid-sized sedan,type_of_vehicle_mpv,type_of_vehicle_others,type_of_vehicle_sports car,type_of_vehicle_stationwagon,type_of_vehicle_suv,type_of_vehicle_truck,...,engine_cap,no_of_owners,depreciation,dereg_value,omv,arf,year,de/coe,model,category
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.234098,0.2,0.049414,0.092296,0.108262,0.092110,0.714286,0.018848,0.194157,0.012966
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.133998,0.2,0.027699,0.040110,0.048767,0.030651,0.666667,0.010721,0.097526,0.020976
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.170189,0.4,0.013361,0.013923,0.033119,0.021471,0.190476,0.004495,0.055992,0.036640
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.094816,0.4,0.009415,0.010496,0.018427,0.011207,0.238095,0.008171,0.023276,0.008224
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.234098,1.0,0.015300,0.007762,0.060777,0.038679,0.142857,0.005537,0.063754,0.008224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21428,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.094716,0.0,0.018400,0.028192,0.023296,0.013970,0.761905,0.009904,0.048331,0.020976
21429,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.094716,0.2,0.013228,0.027313,0.036851,0.021628,0.380952,0.005302,0.105156,0.036640
21430,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.094516,0.2,0.027898,0.010232,0.026831,0.009792,0.571429,0.007520,0.065473,0.028213
21431,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.035095,0.2,0.017488,0.021164,0.027911,0.010646,0.666667,0.006786,0.044265,0.012966


In [49]:
!pip install torch

In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
# 假设 x_final_1 是处理后的特征矩阵，y 是目标变量
# 首先将数据划分为训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x_final_1, y, test_size=0.2, random_state=42)
x_train = x_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
# 将数据转换为 PyTorch 张量
x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)  # 需要将 y 转换为 2D
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# 定义 MLP 模型
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, input_size * 2)
        self.fc2 = nn.Linear(input_size * 2, input_size * 4)
        self.fc3 = nn.Linear(input_size * 4, input_size *2)
        self.fc4 = nn.Linear(input_size * 2, input_size )
        self.fc5 = nn.Linear(input_size, 1)  # 输出层，1个神经元，用于回归
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.relu(self.fc4(x))
        x = self.fc5(x)  # 输出层不需要激活函数（线性激活）
        return x

# 初始化模型
input_size = x_train.shape[1]
model = MLP(input_size)

# 定义损失函数和优化器
criterion = nn.MSELoss()  # 均方误差损失
optimizer = optim.Adam(model.parameters(), lr=0.002)

# 训练模型
num_epochs = 1500  # 设置训练的次数
for epoch in range(num_epochs):
    model.train()  # 设置模型为训练模式
    optimizer.zero_grad()  # 梯度清零
    outputs = model(x_train_tensor)  # 前向传播
    loss = criterion(outputs, y_train_tensor)  # 计算损失
    loss.backward()  # 反向传播
    optimizer.step()  # 更新权重

    if (epoch+1) % 10 == 0:  # 每10个epoch打印一次损失
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 预测和计算 RMSE
model.eval()  # 设置模型为评估模式
with torch.no_grad():  # 关闭梯度计算
    y_pred_tensor = model(x_test_tensor)
    y_pred = y_pred_tensor.numpy()

# 计算 RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"MLP Regressor RMSE (PyTorch): {rmse}")

Epoch [10/1500], Loss: 39896236032.0000
Epoch [20/1500], Loss: 39860932608.0000
Epoch [30/1500], Loss: 39536738304.0000
Epoch [40/1500], Loss: 37799628800.0000
Epoch [50/1500], Loss: 31780833280.0000
Epoch [60/1500], Loss: 22488645632.0000
Epoch [70/1500], Loss: 21304356864.0000
Epoch [80/1500], Loss: 19312396288.0000
Epoch [90/1500], Loss: 17183578112.0000
Epoch [100/1500], Loss: 15111756800.0000
Epoch [110/1500], Loss: 12504520704.0000
Epoch [120/1500], Loss: 9962440704.0000
Epoch [130/1500], Loss: 7786397184.0000
Epoch [140/1500], Loss: 6337134080.0000
Epoch [150/1500], Loss: 5293421568.0000
Epoch [160/1500], Loss: 4538036224.0000
Epoch [170/1500], Loss: 3977049600.0000
Epoch [180/1500], Loss: 3704159744.0000
Epoch [190/1500], Loss: 3396630784.0000
Epoch [200/1500], Loss: 3223708928.0000
Epoch [210/1500], Loss: 3067986944.0000
Epoch [220/1500], Loss: 2862725632.0000
Epoch [230/1500], Loss: 2739182336.0000
Epoch [240/1500], Loss: 2700315904.0000
Epoch [250/1500], Loss: 2615617024.000

In [61]:
# 定义 MLP 模型
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, input_size * 2)
        self.fc2 = nn.Linear(input_size * 2, input_size)
        self.fc3 = nn.Linear(input_size, 1)  # 输出层，1个神经元，用于回归
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)  # 输出层不需要激活函数（线性激活）
        return x

# 初始化模型
input_size = x_train.shape[1]
model_1 = MLP(input_size)

# 定义损失函数和优化器
criterion = nn.MSELoss()  # 均方误差损失
optimizer = optim.Adam(model_1.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.5)
# 训练模型
num_epochs = 2000  # 设置训练的次数
for epoch in range(num_epochs):
    model_1.train()  # 设置模型为训练模式
    optimizer.zero_grad()  # 梯度清零
    outputs = model_1(x_train_tensor)  # 前向传播
    loss = criterion(outputs, y_train_tensor)  # 计算损失
    loss.backward()  # 反向传播
    optimizer.step()  # 更新权重

    if (epoch+1) % 10 == 0:  # 每10个epoch打印一次损失
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 预测和计算 RMSE
model_1.eval()  # 设置模型为评估模式
with torch.no_grad():  # 关闭梯度计算
    y_pred_tensor = model_1(x_test_tensor)
    y_pred = y_pred_tensor.numpy()

# 计算 RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"MLP Regressor RMSE (PyTorch): {rmse}")

Epoch [10/2000], Loss: 39891128320.0000
Epoch [20/2000], Loss: 39832154112.0000
Epoch [30/2000], Loss: 39615578112.0000
Epoch [40/2000], Loss: 39065350144.0000
Epoch [50/2000], Loss: 37945659392.0000
Epoch [60/2000], Loss: 36023713792.0000
Epoch [70/2000], Loss: 33178404864.0000
Epoch [80/2000], Loss: 29604603904.0000
Epoch [90/2000], Loss: 25925615616.0000
Epoch [100/2000], Loss: 23054147584.0000
Epoch [110/2000], Loss: 21515563008.0000
Epoch [120/2000], Loss: 20612808704.0000
Epoch [130/2000], Loss: 19737243648.0000
Epoch [140/2000], Loss: 18841626624.0000
Epoch [150/2000], Loss: 17960732672.0000
Epoch [160/2000], Loss: 17058006016.0000
Epoch [170/2000], Loss: 16122387456.0000
Epoch [180/2000], Loss: 15085091840.0000
Epoch [190/2000], Loss: 14048147456.0000
Epoch [200/2000], Loss: 12944988160.0000
Epoch [210/2000], Loss: 11941657600.0000
Epoch [220/2000], Loss: 10887291904.0000
Epoch [230/2000], Loss: 9923627008.0000
Epoch [240/2000], Loss: 9030770688.0000
Epoch [250/2000], Loss: 826